# 0.1.import libraries

In [ ]:
# basic 
import numpy as np
import pandas as pd
import sys

# ui
import albumentations
import cv2
from tqdm import tqdm

# model training 
import torch
import torch.nn as nn

# model evaluation
from sklearn import metrics

# 0.2.sys.path.append + import 

### tez: a simple pytorch trainer

Idea around tez is simple:

1.keep things as simple as possible

2.make it as customizable as possible

3.clean code

4.faster prototyping

5.production ready

https://pypi.org/project/tez/

In [ ]:
sys.path.append("../input/tez-lib/")
import tez
from tez.callbacks import EarlyStopping

### timm: PyTorch Image Models

https://pypi.org/project/timm/

In [ ]:
sys.path.append("../input/timmmaster/")
import timm

# 0.3 variables

In [ ]:
VALIDATION_FOLD = 0
IMAGE_SIZE = 256
BATCH_SIZE = 64
EPOCHS = 20 

# 1.make dataset 

# 1.1. load data (feature + target)

In [ ]:
df = pd.read_csv("../input/same-old-creating-folds/train_10folds.csv")
train = df[df.kfold != VALIDATION_FOLD].reset_index(drop=True)
valid = df[df.kfold == VALIDATION_FOLD].reset_index(drop=True)

print(df.columns)

dense_features = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
    'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
]

# 1.2. load image path 

In [ ]:
train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in train["Id"].values]
valid_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in valid["Id"].values]

# 1.3. make augmentation

In [ ]:
train_aug = albumentations.Compose(
    [
        albumentations.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1),
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

valid_aug = albumentations.Compose(
    [
        albumentations.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

# 1.4. make dataset (1.1 + 1.2 + 1.3)

In [ ]:
class PawpularDataset:
    def __init__(self, image_paths, dense_features, targets, augmentations):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }

In [ ]:
train_dataset = PawpularDataset(
    image_paths=train_img_paths,
    dense_features=train[dense_features].values,
    targets=train.Pawpularity.values,
    augmentations=train_aug,
)

valid_dataset = PawpularDataset(
    image_paths=valid_img_paths,
    dense_features=valid[dense_features].values,
    targets=valid.Pawpularity.values,
    augmentations=valid_aug,
)


# 2.define model

In [ ]:
class PawpularModel(tez.Model):
    def __init__(self):
        super().__init__()

        self.model = timm.create_model("tf_efficientnet_b0_ns", pretrained=True, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(128 + 12, 1)
        
        self.step_scheduler_after = "epoch"

    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        rmse = metrics.mean_squared_error(targets, outputs, squared=False)
        return {"rmse": rmse}

    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-4)
        return opt

    def forward(self, image, features, targets=None):

        x = self.model(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.out(x)

        if targets is not None:
            loss = nn.MSELoss()(x, targets.view(-1, 1))
            metrics = self.monitor_metrics(x, targets)
            return x, loss, metrics
        return x, 0, {}

# 3.build model

In [ ]:
model = PawpularModel()

es = EarlyStopping(
    monitor="valid_rmse",
    model_path=f"model_f{VALIDATION_FOLD}.bin",
    patience=3,
    mode="min",
    save_weights_only=True,
)

model.fit(
    train_dataset,
    valid_dataset=valid_dataset,
    train_bs=BATCH_SIZE,
    valid_bs=2*BATCH_SIZE,
    device="cuda",
    epochs=EPOCHS,
    callbacks=[es],
    fp16=True,
)